# 1. Extract/Ingest

### 1. Checking Unity Catalog is enabled 
The command found from [here](https://docs.databricks.com/aws/en/data-governance/unity-catalog/get-started#run-a-sql-query-to-confirm-unity-catalog-enablement).

In [0]:
%sql
SELECT CURRENT_METASTORE();

### 2. Bronze class

Methods used in extracting the data:

In [0]:
%python
import requests
# (Py)Spark is intialized in Databricks Python notebooks automatically, thus below imports are redundant if the variable is not overwritten.
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()

class Bronze():
    def __init__(self, dataset_names, urls, catalog, schema):
        '''Setting catalog, schema, creating volume, defining paths'''
        self.dataset_names = dataset_names
        self.urls = urls
        self.catalog = catalog
        self.schema = schema
        self.volume = "raw"
        self.landing_zone = f"/Volumes/{self.catalog}/{self.schema}/{self.volume}"
        self.raw_data_paths = [f"{self.landing_zone}/{data_name}.csv" for data_name in self.dataset_names]
        self.bronze_tables = [f"{data_name}_{self.catalog}" for data_name in dataset_names]

        spark.sql(f"USE {self.catalog}.{self.schema}") # Setting the catalog & schema for the whole notebook
        spark.sql(f"CREATE VOLUME IF NOT EXISTS {self.volume}") # Create volume for raw data landing zone

    def get_response(self, URL):
        '''Reads a URL and returns a requests.Response object'''
        return requests.get(URL, headers={"User-Agent": "Databricks"})

    def df_from_response(self, response, raw_landing_path):
        '''Converts data from requests.Response().content to a PySpark dataframe'''
        response.raise_for_status()
        with open(raw_landing_path, "wb") as f:
            f.write(response.content)
        return spark.read.option("delimiter", ";").csv(raw_landing_path,header=True)

    def df_to_UC_table(self, df, table_path):
        '''Writes a PySpark dataframe to a UC table'''
        df.write.format("delta").mode("overwrite").saveAsTable(table_path)

    def bronze_pipeline(self, URL, raw_landing_path, table_path):
        '''Read data, write to spark dataframe and save as UC table.'''
        print("Reading data from " + URL + "...")
        response = self.get_response(URL)
        print("Creating a Spark dataframe...")
        df = self.df_from_response(response, raw_landing_path)
        print("Saving data to UC table...")
        self.df_to_UC_table(df, table_path)
        return df
    
    def execute_bronze_pipeline(self):
        '''Executes the bronze pipeline for all datasets'''
        for url, land_zone, table_name in zip(self.urls, self.raw_data_paths, self.bronze_tables):
            print("Prosessing the dataset: " + table_name[:-(len(self.catalog)+1)])
            df = self.bronze_pipeline(url, land_zone, table_name)
            display(df.head(5))

### 3. Requesting the data

Defining the data sources; We use the API for open datasets of THL.

In [0]:
# Kaynnit
visits_url = "https://sampo.thl.fi/pivot/prod/fi/avo/perus01/fact_ahil_perus01.csv?row=ammatti-30664&column=aika-87596&column=palvelumuoto-33780#"

# Asiakkaat
customers_url = "https://sampo.thl.fi/pivot/prod/fi/avo/perus01/fact_ahil_perus01.csv?row=ammatti-30664&column=aika-87596&column=palvelumuoto-33780&filter=measure-87454#"

# Kaynnit/Asiakkaat
visits_customers_url = "https://sampo.thl.fi/pivot/prod/fi/avo/perus01/fact_ahil_perus01.csv?row=ammatti-30664&column=aika-87596&column=palvelumuoto-33780&filter=measure-87613#"

urls = [visits_url, customers_url, visits_customers_url]
dataset_names = ["visits", "customers", "visits_customers"]

bronze = Bronze(dataset_names, urls, 'bronze', 'avohilmo')
bronze.execute_bronze_pipeline()